# Advanced prompts

In this notebook, we will explore some advanced prompt patterns:  
+ Few-shots prompt
+ Chain of Thought prompt
+ Program-Aided Language Models (PAL)
+ REACT
+ CO-STAR

## Setup
#### Load the API key and relevant Python libraries.

In [ ]:
!pip install -qU pip
!pip install -qU openai
!pip install -qU google-genai
!pip install -qU python-dotenv

In [2]:
import os
from dotenv import load_dotenv
import openai
from google import genai

load_dotenv("../../.env", override=True)

True

In [5]:
#os.environ["OPENAI_API_KEY"] = "<the key>"
#openai.api_key = os.environ["OPENAI_API_KEY"]
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

model="gpt-5-nano"

In [8]:

def ask(prompt):
    try:
        chat_completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{prompt}"},
            ],
            )
        return(chat_completion.choices[0].message.content)
    except:
        print("An exception has occured.")

### Few-shots prompt

Few-shots prompting means that along with the instruction, you input demonstrations of how the task can be solved. 

##### Zero-shot prompting (without few-shot prompting):

In [7]:
ask("Please analyze the sentiment of the following sentence: 'This phone lasts all day on a single charge. It’s amazing!'")

'- Sentiment: Positive\n- Intensity: High\n- Rationale: The first clause praises battery life ("lasts all day on a single charge"), and the second clause reinforces strong positivity ("It’s amazing!"). Overall, the sentiment is clearly favorable with high intensity.'

##### With few-shot prompting:

In [9]:
query = """
I will provide you with a series of sentences. 
After each sentence, please analyze the sentiment expressed and categorize it as “Positive,” “Negative,” or “Neutral.”

Q) "I absolutely love the friendly service at that restaurant." 
A) {Sentiment: "Positive"}
Q) "The movie was a complete waste of time and money." 
A) {Sentiment: "Negative"}
Q) "It’s a typical day, nothing much happening." 
A) {Sentiment: "Neutral"}

Now, please analyze the sentiment of the following sentence:

"This phone lasts all day on a single charge. It’s amazing!"
"""

ask(query)

'A) {Sentiment: "Positive"}'

### Chain of Thought prompt

The main idea of Chain-of-Thought (CoT) prompting is that LLMs handle complex tasks better if they are divided into intermediate steps.

##### Without Chain of Thought:

In [10]:
query = """
Question: What will be the atomic number of the next element to be added to the periodic table of elements?
"""

ask(query)

'119.\n\nReason: the periodic table currently ends at element 118 (oganesson). The next element would be atomic number 119 if/when it is discovered and officially added. It would have the provisional name ununennium (Uue) until an official name is approved. (Elements 119 and 120 have not yet been confirmed for addition.)'

##### With Chain of Thought:

In [11]:
query = """
- Every element we add to the periodic table of elements has a bigger atomic number comparing to the successor.
- The next element's atomic number adds one to the atomic number of previous one in the periodic table.
- Oganesson, the last element to be added, has the atomic number of 118.

Question: What will be the atomic number of the next element to be added to the periodic table of elements?
"""

ask(query)

'119\n\nReason: Each new element has an atomic number one greater than the previous one. If Oganesson is 118, the next element would be 118 + 1 = 119. Note: as of now, element 119 has not been officially discovered or named; it’s often referred to by the temporary placeholder name Ununennium (Uue).'

### [Program-Aided Language Models (PAL)](https://reasonwithpal.com/)

PAL is a method, where language models generate programs from natural language descriptions, which are then executed by a programmatic runtime like a Python interpreter. That leads to a more accurate result.

##### Without PAL:

In [12]:
ask("There were nine computers in the server room. Five more computers were installed each day, since Monday. How many computers are now in the server room?")

'It depends on how many days have passed since Monday.\n\n- Let k be the number of days from Monday through today, inclusive. Then total computers = 9 + 5k.\n- If today is Friday (Mon, Tue, Wed, Thu, Fri → k = 5), total = 9 + 5×5 = 34.\n- If today is other days, use the corresponding k:\n  - Monday: 9 + 5×1 = 14\n  - Tuesday: 9 + 5×2 = 19\n  - Wednesday: 9 + 5×3 = 24\n  - Thursday: 9 + 5×4 = 29\n\nIf you specify the current day, I can give the exact number.'

##### With PAL:

In [13]:
query = """
There were nine computers in the server room last Monday. 
Five more computers were installed each day, until today. 
How many computers are now in the server room?
create a Python code to answer the question, execute it, and print the result
Explain each step.
"""

print(ask(query))

Here's a small Python solution that computes the number of computers now in the server room, based on the idea that each day after last Monday 5 more computers were added.

What the code does (step by step):
- Step 1: Interpret the problem — last Monday there were 9 computers. Each day after that, including today, 5 more are installed.
- Step 2: Determine how many days have passed from last Monday to today.
- Step 3: Compute total as: total = 9 + 5 * days_passed.
- Step 4: Print the result.

Python code:
import datetime

# Today
today = datetime.date.today()

# Most recent Monday (weekday(): Monday is 0)
last_monday = today - datetime.timedelta(days=today.weekday())

# Days from last Monday to today (0 if today is Monday)
days_passed = (today - last_monday).days

# Initial count and daily increase
initial = 9
daily_increase = 5

# Total computers now
total = initial + days_passed * daily_increase

print("Today:", today)
print("Last Monday:", last_monday)
print("Days since last Monday:"

### [REACT](https://arxiv.org/pdf/2210.03629)

ReAct is a prompting technique that combines reasoning, that  helps the model induce, track, and update action plans as well as handle
exceptions, and actions, that allow it to interface with and gather additional information
from external sources.

##### Without REACT:

In [14]:
query = """
Emma took a 80 minutes plane ride to Seattle. After spending a quarter of hour in a cab, She then took a two hours train ride to Portland, 
but the train was 5 minutes late. Initially, she would only wait for 15 minutes in the train station before the train departs. 
She then took an Uber for the airport, which took 1/2 of hour. Then, after 1 hour waiting in the airport, she took a plane to Vancouver 
that lasted half of the time she spent on the first transportation mode. 
How long did it take her to get to Vancouver?
"""

print(ask(query))

6 hours and 5 minutes.

Reasoning (durations in minutes):
- Plane to Seattle: 80
- Cab: 15
- Train: wait originally 15, but train was 5 minutes late, so actual wait = 20; ride = 120
- Uber to the airport: 30
- Wait at the airport: 60
- Plane to Vancouver: half of the first leg (80) = 40

Total = 80 + 15 + 20 + 120 + 30 + 60 + 40 = 365 minutes = 6 hours 5 minutes.

Note: If you interpret the 5-minute lateness as extending the train ride itself (instead of only delaying departure), you’d get 6 hours 10 minutes. The setup most naturally adds the 5 minutes to the waiting time before departure, yielding 6h5m.


##### With REACT:

In [15]:
query = """
Emma took a 80 minutes plane ride to Seattle. After spending a quarter of hour in a cab, She then took a two hours train ride to Portland, 
but the train was 5 minutes late. Initially, she would only wait for 15 minutes in the train station before the train departs. 
She then took an Uber for the airport, which took 1/2 of hour. Then, after 1 hour waiting in the airport, she took a plane to Vancouver 
that lasted half of the time she spent on the first transportation mode. 
How long did it take her to get to Vancouver? 
Use the REACT framework to show your reasoning.
"""

print(ask(query))

R (Reason)
- List all time segments in order: 
  - Seattle flight: 80 minutes
  - Cab: 15 minutes
  - Portland train: 2 hours (120 minutes), but 5 minutes late
  - Waiting at Portland station: originally 15 minutes, but due to the 5-minute delay, effectively 20 minutes
  - Uber to airport: 30 minutes
  - Waiting at airport: 1 hour (60 minutes)
  - Vancouver flight: half of the first transport time (half of 80 minutes = 40 minutes)

E (Explain)
- The train’s 5-minute lateness adds 5 minutes to the station waiting time, making the wait 20 minutes instead of 15.
- The Vancouver flight duration is determined by the given rule (40 minutes).
- Sum all segments to get total time from the start to arrival in Vancouver.

A (Answer)
- Total time: 80 + 15 + 20 + 120 + 30 + 60 + 40 = 365 minutes = 6 hours and 5 minutes.
- Therefore, it took Emma 6 hours and 5 minutes to get to Vancouver.

C (Check)
- Quick check: 365 minutes seems consistent with the given durations and the lateness adjustment. Al

### CO-STAR  

Effective prompt structuring is crucial for eliciting optimal responses from an LLM. The CO-STAR framework is a handy template for structuring prompts.  

It considers all the key aspects that influence the effectiveness and relevance of an LLM’s response, leading to more optimal responses.  

##### Without CO-STAR:

In [16]:
ask("Write a LinkedIn post to advertise Raiffeisen new product called voiceBot, to allow electronic banking transactions via voice.")

'Raiffeisen is excited to introduce voiceBot, our new voice-enabled banking assistant that lets you perform electronic transactions using just your voice.\n\nWith voiceBot, you can:\n- Check balances and recent activity\n- Transfer funds and pay bills\n- Schedule payments and set up reminders\n- Manage payments hands-free, securely and on your schedule\n\nWhy choose voiceBot?\n- Convenience: bank anytime, anywhere with natural speech\n- Accessibility: designed to be easy to use for all customers\n- Security: voice biometrics, consent-based actions, encryption, and strong authentication\n- Privacy by design: built to meet regulatory standards and your data protection needs\n\nReady to experience voice-powered banking? Sign up for updates and early access here: [URL]\nOr drop a comment with “Interested” and we’ll share more details.\n\n#Raiffeisen #voiceBot #DigitalBanking #VoiceTechnology #FinTech #BankingInnovation #CustomerExperience #Security #PrivacyByDesign'

##### With CO-STAR:

In [17]:
query = """
# CONTEXT #
I want to advertise my company’s new product. My company’s name is Raiffeisen and the product is called VoiceBot, that allow electronic banking transactions via voice.

# OBJECTIVE #
Create a LinkedIn post for me, which aims to get people to click on the link to read it.

# STYLE #
Follow the writing style of successful companies on the same sector that advertise similar products.

# TONE #
Persuasive

# AUDIENCE #
My company’s audience profile on LinkedIn is typically the older generation. Tailor your post to target what this audience typically looks out for in hair products.

# RESPONSE #
The LinkedIn post
"""

ask(query)

'Introducing VoiceBot by Raiffeisen — banking, powered by your voice.\n\nImagine approving a payment with a simple spoken command. VoiceBot makes it possible—secure, easy to use, and built with you in mind.\n\nWhat you can do today:\n- Check your balance and recent activity in seconds\n- Transfer funds or pay bills with natural, spoken prompts\n- Set up alerts for important transactions\n- Use it hands-free at home, in the car, or whenever you prefer not to touch a screen\n\nWhy this matters for you:\n- Clear, straightforward prompts and guidance you can trust\n- Strong security: voice verification plus Raiffeisen’s trusted protections\n- Friendly, accessible support whenever you need it\n\nRead the full story to see how VoiceBot can simplify your daily banking. Click here: https://www.raiffeisen.com/voicebot\n\n#VoiceBanking #Raiffeisen #BankingForAll #Security #Accessibility'